In [ ]:
import os
import tempfile 
from agno.agent import Agent
from agno.models.google import Gemini 
from agno.models.xai import xAI 
from agno.embedder.google import GeminiEmbedder
from agno.tools.duckduckgo import DuckDuckGoTools
from agno.knowledge.pdf import PDFKnowledgeBase, PDFReader 
from agno.vectordb.chroma import ChromaDb
from agno.document.chunking.document import DocumentChunking 


In [ ]:
GOOGLE_API_KEY = ""
PDF_FILE_PATH = "legal_document.pdf"  
CHUNK_SIZE = 1000
OVERLAP = 200

In [ ]:
os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY


In [ ]:
# Initialize Vector DB
vector_db = ChromaDb(
    collection="law",
    path="tmp/chromadb",
    persistent_client=True,
    embedder=GeminiEmbedder()
)

# Load and process PDF
print("Processing document...")

knowledge_base = PDFKnowledgeBase(
    path=PDF_FILE_PATH,
    vector_db=vector_db,
    reader=PDFReader(),
    chunking_strategy=DocumentChunking(chunk_size=CHUNK_SIZE, overlap=OVERLAP)
)

knowledge_base.load(recreate=True, upsert=True)

In [ ]:
# Initialize AI Agents
legal_researcher = Agent(
    name="LegalAdvisor",
    model=Gemini(id="gemini-2.0-flash-exp"),
    knowledge=knowledge_base,
    search_knowledge=True,
    description="Legal Researcher AI - Finds and cites relevant legal cases, regulations, and precedents using all data in the knowledge base.",
    instructions=[
        "Extract all available data from the knowledge base and search for legal cases, regulations, and citations.",
        "If needed, use DuckDuckGo for additional legal references.",
        "Always provide source references in your answers."
    ],
    tools=[DuckDuckGoTools()],
    show_tool_calls=True,
    markdown=True
)

In [ ]:
contract_analyst = Agent(
    name="ContractAnalyst",
    model=Gemini(id="gemini-2.0-flash-exp"),
    knowledge=knowledge_base,
    search_knowledge=True,
    description="Contract Analyst AI - Reviews contracts and identifies key clauses, risks, and obligations using the full document data.",
    instructions=[
        "Extract all available data from the knowledge base and analyze the contract for key clauses, obligations, and potential ambiguities.",
        "Reference specific sections of the contract where possible."
    ],
    show_tool_calls=True,
    markdown=True
)

In [ ]:
legal_strategist = Agent(
    name="LegalStrategist",
    model=Gemini(id="gemini-2.0-flash-exp"),
    knowledge=knowledge_base,
    search_knowledge=True,
    description="Legal Strategist AI - Provides comprehensive risk assessment and strategic recommendations based on all the available data from the contract.",
    instructions=[
        "Using all data from the knowledge base, assess the contract for legal risks and opportunities.",
        "Provide actionable recommendations and ensure compliance with applicable laws."
    ],
    show_tool_calls=True,
    markdown=True
)

In [ ]:
team_lead = Agent(
    name="TeamLead",
    model=Gemini(id="gemini-2.0-flash-exp"),
    description="Team Lead AI - Integrates responses from the Legal Researcher, Contract Analyst, and Legal Strategist into a comprehensive report.",
    instructions=[
        "Combine and summarize all insights provided by the Legal Researcher, Contract Analyst, and Legal Strategist. "
        "Ensure the final report includes references to all relevant sections from the document."
    ],
    show_tool_calls=True,
    markdown=True
)


In [ ]:
# ========== ANALYSIS FUNCTION ==========
def get_team_response(query: str):
    print("Running agents...\n")
    research_response = legal_researcher.run(query)
    contract_response = contract_analyst.run(query)
    strategy_response = legal_strategist.run(query)

    final_response = team_lead.run(
        f"Summarize and integrate the following insights gathered using the full contract data:\n\n"
        f"Legal Researcher:\n{research_response}\n\n"
        f"Contract Analyst:\n{contract_response}\n\n"
        f"Legal Strategist:\n{strategy_response}\n\n"
        "Provide a structured legal analysis report that includes key terms, obligations, risks, and recommendations, with references to the document."
    )
    return final_response

In [ ]:
print("Select analysis type:")
options = [
    "Contract Review",
    "Legal Research",
    "Risk Assessment",
    "Compliance Check",
    "Custom Query"
]
for idx, opt in enumerate(options):
    print(f"{idx + 1}. {opt}")

In [ ]:
selected = int(input("Enter your choice (1-5): "))
if selected == 5:
    query = input("Enter your custom legal question: ")
else:
    predefined_queries = {
        1: (
            "Analyze this document, contract, or agreement using all available data from the knowledge base. "
            "Identify key terms, obligations, and risks in detail."
        ),
        2: (
            "Using all available data from the knowledge base, find relevant legal cases and precedents related to this document, contract, or agreement. "
            "Provide detailed references and sources."
        ),
        3: (
            "Extract all data from the knowledge base and identify potential legal risks in this document, contract, or agreement. "
            "Detail specific risk areas and reference sections of the text."
        ),
        4: (
            "Evaluate this document, contract, or agreement for compliance with legal regulations using all available data from the knowledge base. "
            "Highlight any areas of concern and suggest corrective actions."
        ),
    }
    query = predefined_queries[selected]

In [ ]:
response = get_team_response(query)

In [ ]:
print(response.content if response.content else "No response generated.")

In [ ]:
key_points_response = team_lead.run(
    f"Summarize the key legal points from this analysis:\n{response.content}"
)
print(key_points_response.content if key_points_response.content else "No summary generated.")

In [ ]:
recommendations_response = team_lead.run(
    f"Provide specific legal recommendations based on this analysis:\n{response.content}"
)
print(recommendations_response.content if recommendations_response.content else "No recommendations generated.")